Sample Average Approximation for the stochastic knapsack problem

In [1]:
import numpy as np
import pandas as pd

In [2]:
df3 = pd.read_excel (r'knapsack.xlsx')

In [3]:
w = pd.DataFrame(df3.iloc[1:6, 2:12].values)
r = df3.iloc[10:16, 2].values
q = df3.iloc[7,2]
c = df3.iloc[8,2]

In [4]:
from pyomo.environ import *

In [5]:
# Create model
m = ConcreteModel()

In [6]:
# SETS ######################

m.i = Set(initialize = np.arange(w.shape[0]))
#m.k.pprint()

m.j = Set(initialize = np.arange(w.shape[1]))
#m.n.pprint()

## PARAMETERS ###############

m.N = Param(initialize = w.shape[1])

# Transform matrix to dictionary
w.to_dict()
w = w.stack().to_dict()
r = dict(enumerate(r))

m.w = Param(m.i, m.j, initialize = w)
#m.w.pprint()

m.r = Param(m.i, initialize = r)
#m.r.pprint()

m.q = Param(initialize = q)
#m.q.pprint()

m.c = Param(initialize = c)
#m.c.pprint()

## VARIABLES ################

m.x = Var(m.i, within = Binary, initialize = 0)
#m.x.pprint()

m.z = Var(m.j, within =NonNegativeReals, bounds = (0, None), initialize = 0)
#m.z.pprint()

## CONSTRAINTS ##############

def penalties(model, j):
    return(m.z[j] >= (sum(m.w[i,j] * m.x[i] for i in m.i) - m.q))
m.pen = Constraint(m.j, rule = penalties)
#m.pen.pprint()

## OBJECTIVE ################

def objective(model):
    return(sum(m.r[i] * m.x[i] for i in m.i) - m.c/m.N * sum(m.z[j] for j in m.j))
m.obj = Objective(rule = objective, sense = maximize)

In [7]:
from pyomo.opt import SolverFactory
import pyomo.environ

opt = SolverFactory('gurobi')
opt.options['IntFeasTol']= 10e-10
opt.options['MIPGap'] = 0

results = opt.solve(m, load_solutions=True, tee = True)

Using license file /Users/fietekrutein/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/3f/ty6j5m0x59l2j7rp1ktlmbx80000gn/T/tmpzz_9j9xq.pyomo.lp
Reading time = 0.00 seconds
x16: 11 rows, 16 columns, 61 nonzeros
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Changed value of parameter MIPGap to 0.0
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 11 rows, 16 columns and 61 nonzeros
Model fingerprint: 0x2bd6ab04
Variable types: 11 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros
Variable types: 10 continuous, 5 integer (5 binary)

Root relaxation: objective 2.26